In [69]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc

fiveFiftyPath = r"C:\Users\crmai\OneDrive\Documents\GT Spring 2025\BMED 6517\Project1_updated_021125\5_mr_50_cond\simulated_noNoise.txt"
fourtyFiftyPath = r"C:\Users\crmai\OneDrive\Documents\GT Spring 2025\BMED 6517\Project1_updated_021125\5_mr_50_cond\simulated_noNoise.txt"
hundidHundidPath = r"C:\Users\crmai\OneDrive\Documents\GT Spring 2025\BMED 6517\Project1_updated_021125\5_mr_50_cond\simulated_noNoise.txt"


data_5_mr_50_cond = np.loadtxt(fiveFiftyPath)
np.save(r"C:\Users\crmai\OneDrive\Documents\GT Spring 2025\BMED 6517\Project1_updated_021125\5_mr_50_cond\simulated_noNoiseAll", data_5_mr_50_cond)
data_5_mr_50_cond_tfs = data_5_mr_50_cond[1:,:100]
np.save(r"C:\Users\crmai\OneDrive\Documents\GT Spring 2025\BMED 6517\Project1_updated_021125\5_mr_50_cond\simulated_noNoiseTFs", data_5_mr_50_cond_tfs)
data_5_mr_50_cond_tgs = data_5_mr_50_cond[1:,100:200]
np.save(r"C:\Users\crmai\OneDrive\Documents\GT Spring 2025\BMED 6517\Project1_updated_021125\5_mr_50_cond\simulated_noNoiseTGs", data_5_mr_50_cond_tgs)

data_40_mr_50_cond = np.loadtxt(fourtyFiftyPath)
np.save(r"C:\Users\crmai\OneDrive\Documents\GT Spring 2025\BMED 6517\Project1_updated_021125\40_mr_50_cond\simulated_noNoiseAll", data_40_mr_50_cond)
data_40_mr_50_cond_tfs = data_40_mr_50_cond[1:,:100]
np.save(r"C:\Users\crmai\OneDrive\Documents\GT Spring 2025\BMED 6517\Project1_updated_021125\40_mr_50_cond\simulated_noNoiseTFs", data_40_mr_50_cond_tfs)
data_40_mr_50_cond_tgs = data_40_mr_50_cond[1:,100:200]
np.save(r"C:\Users\crmai\OneDrive\Documents\GT Spring 2025\BMED 6517\Project1_updated_021125\40_mr_50_cond\simulated_noNoiseTGs", data_40_mr_50_cond_tgs)

data_100_mr_100_cond = np.loadtxt(hundidHundidPath)
np.save(r"C:\Users\crmai\OneDrive\Documents\GT Spring 2025\BMED 6517\Project1_updated_021125\100_mr_100_cond\simulated_noNoiseAll", data_100_mr_100_cond)
data_100_mr_100_cond_tfs = data_100_mr_100_cond[1:,:100]
np.save(r"C:\Users\crmai\OneDrive\Documents\GT Spring 2025\BMED 6517\Project1_updated_021125\100_mr_100_cond\simulated_noNoiseTFs", data_100_mr_100_cond_tfs)
data_100_mr_100_cond_tgs = data_100_mr_100_cond[1:,100:200]
np.save(r"C:\Users\crmai\OneDrive\Documents\GT Spring 2025\BMED 6517\Project1_updated_021125\100_mr_100_cond\simulated_noNoiseTGs", data_100_mr_100_cond_tgs)


In [70]:
def create_ground_truth_matrix(ground_truth_file):
    """
    Create a ground truth adjacency matrix from the edge list file.
    
    Parameters:
    -----------
    ground_truth_file : str
        Path to the ground truth file (bipartite_GRN.csv)
        
    Returns:
    --------
    numpy.ndarray
        Binary adjacency matrix representing the ground truth GRN
    """
    # Read ground truth GRN
    ground_truth_edges = np.loadtxt(ground_truth_file, delimiter=',', dtype=int)
    
    # Create ground truth adjacency matrix
    n_tf = 100
    n_g = 100
    ground_truth_matrix = np.zeros((n_tf, n_g), dtype=int)
    
    # Fill ground truth adjacency matrix
    for edge in ground_truth_edges:
        tf_id = int(edge[0])  # TF id 
        gene_id = int(edge[1]) - 100  # Target gene id, adjusted for matrix indexing
        
        # Ensure indices are within bounds
        if 0 <= tf_id < n_tf and 0 <= gene_id < n_g:
            ground_truth_matrix[tf_id, gene_id] = 1
        else:
            print(f"Warning: Edge {tf_id} -> {gene_id+100} is out of bounds and will be ignored")
    
    # Print some statistics
    num_edges = np.sum(ground_truth_matrix)
    print(f"Ground truth matrix created: {ground_truth_matrix.shape}")
    print(f"Number of regulatory interactions: {num_edges}")
    print(f"Average number of TFs regulating each gene: {num_edges/n_g:.2f}")
    print(f"Average number of genes regulated by each TF: {num_edges/n_tf:.2f}")
    
    return ground_truth_matrix

In [71]:
def run_grn_reconstruction(expression_file, ground_truth_file, threshold=0.5):
    """
    Run the complete GRN reconstruction pipeline.
    
    Parameters:
    -----------
    expression_file : str
        Path to the expression data file
    ground_truth_file : str
        Path to the ground truth file
    threshold : float
        Correlation threshold
    
    Returns:
    --------
    dict with results and metrics
    """
    # Load expression data
    print("Loading expression data...")
    expression_data = np.loadtxt(expression_file)
    print(f"Expression data shape: {expression_data.shape}")
    
    # Extract header and values
    gene_ids = expression_data[0, :].astype(int)
    exp_values = expression_data[1:, :]
    
    # Verify data structure
    print(f"Gene IDs range: {gene_ids.min()} to {gene_ids.max()}")
    print(f"Number of conditions: {exp_values.shape[0]}")
    print(f"First few values:\n{exp_values[:3, :5]}")
    
    # Load ground truth
    print("\nLoading ground truth...")
    #ground_truth_matrix = create_ground_truth_matrix(ground_truth_file)
    def create_ground_truth_matrix(ground_truth_file):

        # Read ground truth GRN
        ground_truth_edges = np.loadtxt(ground_truth_file, delimiter=',', dtype=int)
    
        # Create ground truth adjacency matrix
        n_tf = 100
        n_g = 100
        ground_truth_matrix = np.zeros((n_tf, n_g), dtype=int)
    
        # Fill ground truth adjacency matrix
        for edge in ground_truth_edges:
            tf_id = int(edge[0])  # TF id 
            gene_id = int(edge[1]) - 100  # Target gene id, adjusted for matrix indexing
        
            # Ensure indices are within bounds
            if 0 <= tf_id < n_tf and 0 <= gene_id < n_g:
                ground_truth_matrix[tf_id, gene_id] = 1
            else:
                print(f"Warning: Edge {tf_id} -> {gene_id+100} is out of bounds and will be ignored")
    
        # Print some statistics
        num_edges = np.sum(ground_truth_matrix)
        print(f"Ground truth matrix created: {ground_truth_matrix.shape}")
        print(f"Number of regulatory interactions: {num_edges}")
        print(f"Average number of TFs regulating each gene: {num_edges/n_g:.2f}")
        print(f"Average number of genes regulated by each TF: {num_edges/n_tf:.2f}")
    
        return ground_truth_matrix
    
    ground_truth_matrix = create_ground_truth_matrix(ground_truth_file)
    # Extract TF and target gene expression
    tf_data = exp_values[:, :100]
    target_data = exp_values[:, 100:200]
    
    # Standardize data
    print("\nStandardizing data...")
    scaler = StandardScaler()
    tf_data_scaled = scaler.fit_transform(tf_data)
    
    scaler = StandardScaler()
    target_data_scaled = scaler.fit_transform(target_data)
    
    # Calculate correlations
    print("\nCalculating correlations...")
    correlation_matrix = np.zeros((100, 100))
    for i in range(100):
        for j in range(100):
            correlation_matrix[i, j] = np.corrcoef(tf_data_scaled[:, i], 
                                                 target_data_scaled[:, j])[0, 1]
    
    # Create adjacency matrix
    print(f"\nCreating adjacency matrix with threshold {threshold}...")
    adjacency_matrix = np.zeros((100, 100), dtype=int)
    adjacency_matrix[abs(correlation_matrix) >= threshold] = 1
    
    
    
    return {
        'correlation_matrix': correlation_matrix,
        'adjacency_matrix': adjacency_matrix,
        'ground_truth_matrix': ground_truth_matrix,
        #'metrics': metrics
    }

In [72]:
def evaluate_grn_accuracy(predicted_adjacency, ground_truth_adjacency):
    # Calculate true positives, false positives, true negatives, false negatives
    tp = np.sum((predicted_adjacency == 1) & (ground_truth_adjacency == 1))
    fp = np.sum((predicted_adjacency == 1) & (ground_truth_adjacency == 0))
    tn = np.sum((predicted_adjacency == 0) & (ground_truth_adjacency == 0))
    fn = np.sum((predicted_adjacency == 0) & (ground_truth_adjacency == 1))
    
    # Calculate metrics
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    #return accuracy, precision, recall, f1_score, tp, fp, tn, fn

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'true_positives': tp,
        'false_positives': fp,
        'true_negatives': tn,
        'false_negatives': fn
    }

In [74]:
if __name__ == "__main__":
    # Load ground truth
    ground_truth_path = r"C:\Users\crmai\OneDrive\Documents\GT Spring 2025\BMED 6517\Project1_updated_021125\100_mr_100_cond\bipartite_GRN.csv"
    #ground_truth_matrix = create_ground_truth_matrix(ground_truth_path)

    # Generate predictions using correlation or other methods
    #predicted_matrix = build_improved_grn_adjacency_matrix(data_100_mr_100_cond, threshold=0.5)
    
    results = run_grn_reconstruction(hundidHundidPath, ground_truth_path, threshold=0.3)
    correlation_matrix = results['correlation_matrix']
    adjacency_matrix = results['adjacency_matrix']
    ground_truth_matrix = results['ground_truth_matrix']

    # Evaluate performance
    print("\nEvaluating performance...")
    metrics = evaluate_grn_accuracy(adjacency_matrix, ground_truth_matrix)

    # Print results
    '''print("\nGRN Reconstruction Results:")
    print("\naccuracy: ", {accuracy})
    print("\nprecision: ", {precision})
    print("\nrecall: ", {recall})
    print("\nf1: ", {f1_score})
    print({tp})
    print({fp})
    print({tn})
    print({fn})'''
    print("GRN Reconstruction Results:")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1_score']:.4f}")
    print(f"True Positives: {metrics['true_positives']}")
    print(f"False Positives: {metrics['false_positives']}")
    print(f"True Negatives: {metrics['true_negatives']}")
    print(f"False Negatives: {metrics['false_negatives']}")


Loading expression data...
Expression data shape: (51, 200)
Gene IDs range: 0 to 199
Number of conditions: 50
First few values:
[[ 3.52  7.57  6.78  4.53  6.94]
 [ 4.77  2.59 10.5   6.69  5.19]
 [ 2.02  2.51  7.57  3.86  4.3 ]]

Loading ground truth...
Ground truth matrix created: (100, 100)
Number of regulatory interactions: 487
Average number of TFs regulating each gene: 4.87
Average number of genes regulated by each TF: 4.87

Standardizing data...

Calculating correlations...

Creating adjacency matrix with threshold 0.3...

Evaluating performance...
GRN Reconstruction Results:
Accuracy: 0.4134
Precision: 0.0505
Recall: 0.6201
F1 Score: 0.0934
True Positives: 302
False Positives: 5681
True Negatives: 3832
False Negatives: 185
